In [ ]:
import os
import pandas as pd
from ast import literal_eval
from bs4 import BeautifulSoup
import requests
import urllib.request
import json
from datetime import datetime

Role: https://www.gov.uk/api/content/government/ministers/attorney-general

People: https://www.gov.uk/api/content/government/people/matthew-hancock and list of all people: https://www.gov.uk/api/content/government/people

In [2]:
DATA_DIR = os.getenv("DATA_DIR")

## Extract people-related content and metadata

In [38]:
def extract_html_links(text):
    """
    Grab any GOV.UK domain-specific (people) links from page text.
    :param text: Text within a details sub-section, refer to filtered for keys.
    :return: list of links
    """
    links = []
    try:
        soup = BeautifulSoup(text, "html.parser")
        links = [link.get('href') for link in soup.findAll('a', href=True)]
    except Exception:
        print("error")
    return [l.replace("https://www.gov.uk/", "/") for l in links
            if l.startswith("/government/people")]

In [ ]:
# ## trial
# "https://www.gov.uk/government/people"
# url = requests.get("https://www.gov.uk/government/people")
# htmltext = url.text
# int("https://www.gov.uk/government/people?page=2".split("=")[-1])

In [39]:
all_people = []
ind = 1
next_page = 1
while ind == next_page:
    target = "https://www.gov.uk/government/people?page={}".format(ind)
    url = requests.get(target)
    link_list = extract_html_links(url.text)
    all_people.extend(link_list[0:-1])
    ind+=1
    next_page = int(link_list[-1].split("=")[-1])
    if ind%10==0:
        print(f"at index: {ind}")

at: 10
at: 20
at: 30
at: 40
at: 50
at: 60
at: 70


In [36]:
with open(os.path.join(DATA_DIR, "people_urls.csv"), "w") as write:
    for p in all_people:
        write.write(p+"\n")

### Use people list to call content api and store relevant content

In [ ]:
people_content = []
not_found = []
for i, people_url in enumerate(all_people):
    try:
        url = "https://www.gov.uk/api/content" + people_url
        content_item = json.loads(urllib.request.urlopen(url).read())
        people_content.append(content_item)
    except Exception:
        # logger.debug("Url \'{}\' not found".format(url))
        not_found.append(people_url)
    if i%500==0:
        print(f'at: {i}')
        
df = pd.DataFrame(people_content)
df.to_csv(os.path.join(DATA_DIR, "people_content_store.csv"), index=False)

at: 0
at: 500


In [ ]:
len(not_found)

### Preprocess appointment data in people content store data